In [77]:
# ================================
# Install dependencies
# ================================
%pip install --quiet yfinance pandas numpy scikit-learn tensorflow


In [78]:

# -----------------------------
# Imports
# -----------------------------
import os
import json
import concurrent.futures
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from tqdm import tqdm
import inspect


In [79]:

# -----------------------------
# Parameters
# -----------------------------
SEQ_LEN = 20
EPOCHS = 20
BATCH_SIZE = 32
MAX_WORKERS = 8
DATA_HISTORY_IN_YEARS = 20
DATA_INTERVAL_IN_DAYS = 1
DATA_DIR = "data"
FEATURES = ["Open", "High", "Low", "Close", "Volume"]
MODEL_NAME = "nifty100_lstm_model_v2.keras"
DATA_DICT_FILE = "data_dict.json"
os.makedirs(DATA_DIR, exist_ok=True)


In [80]:
# -----------------------------
# Training Stocks
# -----------------------------
symbols = [
    "ADANIPORTS.NS", "ASIANPAINT.NS", "AXISBANK.NS", "BAJAJ-AUTO.NS", "BAJFINANCE.NS",
    "BAJAJFINSV.NS", "BPCL.NS", "BHARTIARTL.NS", "BRITANNIA.NS", "CIPLA.NS",
    "COALINDIA.NS", "DRREDDY.NS", "EICHERMOT.NS", "GAIL.NS", "GRASIM.NS",
    "HCLTECH.NS", "HDFCBANK.NS", "HDFCLIFE.NS", "ICICIBANK.NS", "INDUSINDBK.NS",
    "INFY.NS", "ITC.NS", "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS",
    "M&M.NS", "MARUTI.NS", "NTPC.NS", "NESTLEIND.NS", "ONGC.NS",
    "POWERGRID.NS", "RELIANCE.NS", "SBILIFE.NS", "SBIN.NS", "SUNPHARMA.NS",
    "TCS.NS", "TATACONSUM.NS", "TATASTEEL.NS", "TECHM.NS", "TITAN.NS",
    "ULTRACEMCO.NS", "UPL.NS", "WIPRO.NS", "DIVISLAB.NS"
]


In [81]:

# -----------------------------
# Data Fetcher (cached download)
# -----------------------------

def fetch_stock(sym):
    file_path = os.path.join(DATA_DIR, f"{sym}.csv")

    if os.path.exists(file_path):
        # Load existing CSV
        df_existing = pd.read_csv(file_path, parse_dates=["Date"])

        # Find the last available date
        last_date = df_existing["Date"].max()

        start_date = (last_date + pd.Timedelta(days=1)).strftime("%Y-%m-%d")

        # if start_date is greater than today then return existing df
        if pd.to_datetime(start_date) > pd.to_datetime("today"):
            return sym, df_existing


        # Fetch new data starting from the day after last_date
        df_new = yf.download(
            sym,
            start=(last_date + pd.Timedelta(days=1)).strftime("%Y-%m-%d"),
            interval=f"{DATA_INTERVAL_IN_DAYS}d"
        )

        if not df_new.empty:
            df_new = df_new[FEATURES].dropna()

            # Flatten MultiIndex if present
            if isinstance(df_new.columns, pd.MultiIndex):
                df_new.columns = df_new.columns.get_level_values(0)

            df_new = df_new.reset_index()

            # Append only new rows
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            df_combined = df_combined.drop_duplicates(subset=["Date"]).reset_index(drop=True)

            # Save updated CSV
            df_combined.to_csv(file_path, index=False)
            return sym, df_combined

        # If no new data, return existing
        return sym, df_existing

    else:
        # No CSV yet — fetch full history
        df = yf.download(
            sym,
            period=f"{DATA_HISTORY_IN_YEARS}y",
            interval=f"{DATA_INTERVAL_IN_DAYS}d"
        )
        if df.empty:
            return sym, None

        df = df[FEATURES].dropna()

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        df = df.reset_index()
        df.to_csv(file_path, index=False)
        return sym, df


In [55]:

# -----------------------------
# Feature Engineering
# -----------------------------
def add_features(df: pd.DataFrame, drop_date: bool = False):
    df = df.copy()
    df["Return"] = df["Close"].pct_change()
    df["MA7"] = df["Close"].rolling(7).mean()
    df["MA21"] = df["Close"].rolling(21).mean()
    df["STD21"] = df["Close"].rolling(21).std()
    df["Upper_BB"] = df["MA21"] + (df["STD21"] * 2)
    df["Lower_BB"] = df["MA21"] - (df["STD21"] * 2)
    df["EMA"] = df["Close"].ewm(span=20, adjust=False).mean()
    df["Momentum"] = df["Close"] - df["Close"].shift(5)
    if drop_date:
        df = df.drop(columns=["Date"])
    return df.dropna()


In [82]:

# -----------------------------
# Sequence Creator
# -----------------------------
def create_sequences(data, seq_len=SEQ_LEN, split=0.8):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
        y.append(data[i, 3])  # 3 = Close column
    X, y = np.array(X), np.array(y).reshape(-1,1)
    split_idx = int(len(X) * split)
    return (X[:split_idx], y[:split_idx]), (X[split_idx:], y[split_idx:])


In [83]:

# -----------------------------
# Build Model
# -----------------------------
def build_model(input_shape):
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=input_shape),
        Dropout(0.4),
        LSTM(64),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss=Huber())
    return model


In [ ]:

# -----------------------------
# Training Pipeline
# -----------------------------

def save_data_dict(data_dict, file_path=DATA_DICT_FILE):
    # Convert DataFrames to a JSON-friendly format (dictionary of lists)
    json_data_dict = {}
    for symbol, df in data_dict.items():
        # Convert DataFrame to dictionary of lists, handling datetimes
        json_data_dict[symbol] = df.to_dict(orient='list')
        # Convert datetime objects to strings for JSON compatibility
        if 'Date' in json_data_dict[symbol]:
            json_data_dict[symbol]['Date'] = [str(date) for date in json_data_dict[symbol]['Date']]
    # Dump the dictionary to a JSON file
    with open(file_path, "w") as f:
        json.dump(json_data_dict, f, indent=4)
    print(f"✅ Saved updated data_dict to {file_path}")

def load_data_dict(file_path=DATA_DICT_FILE):
    if not os.path.exists(file_path):
        return {}
    with open(file_path, "r") as f:
        loaded = json.load(f)
    data_dict = {}
    for sym, data in loaded.items():
        df = pd.DataFrame(data)
        if "Date" in df.columns:
            df["Date"] = pd.to_datetime(df["Date"])
        data_dict[sym] = df
    print(f"✅ Loaded existing data_dict from {file_path}")
    return data_dict

def update_data_dict(symbols):
    # Load existing dictionary if present
    data_dict = load_data_dict()

    # Fetch in parallel
    with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(tqdm(executor.map(fetch_stock, symbols), total=len(symbols), desc="Fetching data"))

    for sym, df in results:
        if df is not None:
            if sym in data_dict:
                # Append only new rows
                df_existing = data_dict[sym]
                df_combined = pd.concat([df_existing, df], ignore_index=True)
                df_combined = df_combined.drop_duplicates(subset=["Date"]).reset_index(drop=True)
                data_dict[sym] = add_features(df_combined)
            else:
                data_dict[sym] = add_features(df)

    # Save back
    save_data_dict(data_dict)
    return data_dict


data_dict = update_data_dict(symbols)

✅ Loaded existing data_dict from data_dict.json


Fetching data: 100%|██████████| 44/44 [00:00<00:00, 63.84it/s]


✅ Saved updated data_dict to data_dict.json


In [85]:
# -----------------------------
# Scale the Data
# -----------------------------

scaler_dict, scaled_data_dict = {}, {}
for sym, df in data_dict.items():
    df = df.drop(columns=["Date"])
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    scaler_dict[sym] = scaler
    scaled_data_dict[sym] = scaled


In [86]:
# -----------------------------
# Prepare Training Data
# -----------------------------
X_train_list, y_train_list, X_test_list, y_test_list = [], [], [], []
for scaled in scaled_data_dict.values():
    (X_tr, y_tr), (X_te, y_te) = create_sequences(scaled, SEQ_LEN)
    X_train_list.append(X_tr); y_train_list.append(y_tr)
    X_test_list.append(X_te);  y_test_list.append(y_te)

X_train, y_train = np.vstack(X_train_list), np.vstack(y_train_list)
X_test, y_test = np.vstack(X_test_list), np.vstack(y_test_list)


In [87]:
# -----------------------------
# Build Model
# -----------------------------

model = build_model((SEQ_LEN, X_train.shape[2]))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [27]:
# -----------------------------
# Fit or Train the model
# -----------------------------

early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-4, verbose=1)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=EPOCHS, batch_size=BATCH_SIZE,
    callbacks=[early_stopping, lr_scheduler]
)

# -----------------------------
# Save Model
# -----------------------------

model.save(MODEL_NAME)


Epoch 1/20
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 200s 48ms/step - loss: 5.0093e-04 - val_loss: 1.0580e-04 - learning_rate: 0.0010
Epoch 2/20
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 198s 48ms/step - loss: 1.4614e-04 - val_loss: 1.1870e-04 - learning_rate: 0.0010
Epoch 3/20
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 200s 47ms/step - loss: 1.2351e-04 - val_loss: 3.3628e-05 - learning_rate: 0.0010
Epoch 4/20
4132/4133 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 1.1809e-04
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 202s 47ms/step - loss: 1.1809e-04 - val_loss: 2.9194e-05 - learning_rate: 0.0010
Epoch 5/20
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 190s 46ms/step - loss: 9.8996e-05 - val_loss: 1.7921e-05 - learning_rate: 5.0000e-04
Epoch 6/20
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 193s 47ms/step - loss: 9.5710e-05 - val_loss: 3.8067e-05 - learning_rate: 5.0000e-04
Epoch 7/20
4132/4133 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 9.8631e-05
Epoch 7: ReduceLROnPlateau reducing learn

In [88]:


# -----------------------------
# Evaluate
# -----------------------------
model = load_model(MODEL_NAME, compile=False)
y_pred = model.predict(X_test)

# fix RMSE for old sklearn
if "squared" in inspect.signature(mean_squared_error).parameters:
    rmse = mean_squared_error(y_test, y_pred, squared=False)
else:
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")


1293/1293 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step
RMSE: 0.0330, MAE: 0.0116, R²: 0.9827


In [92]:
def predict_stock_simple(symbol, model_path=MODEL_NAME):
    """Simplified approach - retrain scaler on recent data"""
    model = load_model(model_path, compile=False)

    # Fetch data
    sym, df = fetch_stock(symbol)
    if df is None:
        print(f"❌ No data for {symbol}")
        return None

    # Apply feature engineering
    df_features = add_features(df, drop_date=True)

    # Use only recent data for scaling (e.g., last 200 days)
    recent_data = df_features.tail(200)

    # Create a fresh scaler on recent data
    scaler = MinMaxScaler()
    scaled_recent = scaler.fit_transform(recent_data)

    # Get last SEQ_LEN rows for prediction
    X = scaled_recent[-SEQ_LEN:].reshape(1, SEQ_LEN, scaled_recent.shape[1])

    # Predict
    pred_scaled = model.predict(X, verbose=0)[0][0]

    # Inverse transform
    last_row = scaled_recent[-1].copy()
    last_row[3] = pred_scaled
    pred_actual = scaler.inverse_transform([last_row])[0, 3]

    last_close = df_features["Close"].iloc[-1]

    return {
        "symbol": symbol,
        "last_close": float(last_close),
        "predicted_close": float(pred_actual)
    }

In [93]:
print(predict_stock_simple("TCS.NS"))

{'symbol': 'TCS.NS', 'last_close': 3020.89990234375, 'predicted_close': 3017.987731115834}
